In [1]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [2]:
sp = SparkSession.builder.appName("Checker").getOrCreate()
sp

22/10/07 21:00:59 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 172.19.34.232 instead (on interface eth0)
22/10/07 21:00:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 21:01:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
m_fraud = sp.read.option("inferScehma", True).parquet("../data/curated/merchant_fraud")
m_fraud.show(1)

+-------+-------------------+-----------------+
|user_id|     order_datetime|fraud_probability|
+-------+-------------------+-----------------+
|   6228|2021-12-19 00:00:00|         97.62981|
+-------+-------------------+-----------------+
only showing top 1 row



In [4]:
merchants= sp.read.option("inferSchema", True).option("header", True).csv("../data/tables/merchants_tbl_processed.csv")
merchants.show(1)

+-------------+------------+--------------------+--------------+--------+
|         name|merchant_abn|         Description|Earnings_Class|BNPL_Fee|
+-------------+------------+--------------------+--------------+--------+
|Felis Limited| 10023283211|furniture home fu...|             e|    0.18|
+-------------+------------+--------------------+--------------+--------+
only showing top 1 row



### CHECKING NEW PROCESSED

In [4]:
transactions = sp.read.option("inferScehma", True).parquet("../data/processed/transactions")
transactions.show(1)

+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
|order_id|user_id|merchant_abn|dollar_value|order_datetime|Natural_var|Potential_Outlier|holiday|dayofmonth|month|dayofweek|
+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
|       3|      3| 60956456424|      136.68|    2021-08-20|          0|                0|      0|        20|    8|        6|
+--------+-------+------------+------------+--------------+-----------+-----------------+-------+----------+-----+---------+
only showing top 1 row



In [5]:
merchants_processed = sp.read.option("inferScehma", True).parquet("../data/processed/merchants")
merchants_processed.show(1)

+------------+-------------+--------------+--------+----+---------------+---------------+----------------+-----------------+
|merchant_abn|         name|Earnings_Class|BNPL_Fee|tags|avg_monthly_inc|monthly_entropy|postcode_entropy|          revenue|
+------------+-------------+--------------+--------+----+---------------+---------------+----------------+-----------------+
| 10023283211|Felis Limited|             e|    0.18| 0.0|     -0.0952381|      2.9858725|        7.439226|703277.8708539009|
+------------+-------------+--------------+--------+----+---------------+---------------+----------------+-----------------+
only showing top 1 row



In [8]:
merchants_processed.select("Description").distinct().count()

25

In [13]:
lookup = sp.read.option("inferSchema", True).csv("../data/tables/description_lookup.csv")
lookup = lookup.withColumnRenamed("_c0", "Description").withColumnRenamed("_c1", "tag_number")
lookup.show(1)

+--------------------+----------+
|         Description|tag_number|
+--------------------+----------+
|furniture home fu...|       0.0|
+--------------------+----------+
only showing top 1 row



In [15]:
merchants_processed.select("Description").distinct().count()

25

In [16]:
merchants.select("Description").distinct().count()

25

In [17]:
new_merchants = merchants.join(lookup, on="Description")
new_merchants.show(1)

+--------------------+-------------+------------+--------------+--------+----------+
|         Description|         name|merchant_abn|Earnings_Class|BNPL_Fee|tag_number|
+--------------------+-------------+------------+--------------+--------+----------+
|furniture home fu...|Felis Limited| 10023283211|             e|    0.18|       0.0|
+--------------------+-------------+------------+--------------+--------+----------+
only showing top 1 row

